In [1]:
# import module written for processing our gene mapping to gene ID, filter with e-value, pident
from network_analysis.map_string_to_ID import *

# node_path referes to diamond blastp --in REPRESENTING GENE --db string.dmnd
node_path = '/home/hermuba/nas2/resist_subnet/mentha/node_mapper.blast'
node = read_node_to_df(node_path)



In [2]:
# select nodes with high bitscore with string 511145 proteins
selected_nodes = filter_node(node)

/home/hermuba/resistanceExp/network_analysis/map_string_to_ID.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_nodes.drop_duplicates(subset = ['sseqid'], inplace = True)


In [3]:
selected_nodes.shape

(1563, 3)

In [4]:
selected_nodes.reset_index(inplace = True)

In [5]:
selected_nodes = selected_nodes.drop_duplicates('sseqid')

In [6]:
# read the edge
edge_path = '/home/hermuba/data0118/map_to_exist_net/mentha_ecoli'
edge = pd.read_csv(edge_path, sep = ';')

In [7]:
edge.head()

,Protein A,Gene A,Protein B,Gene B,Score,PMID
0,P26607,P26607,P07027,UVRY,0.332,15522865 11022030
1,P26602,UBIC,P26602,UBIC,0.210,15690043 24561554
2,P76316,DCYD,P60422,RPLB,0.236,19402753
3,P60422,RPLB,P0A8M6,YEEX,0.236,19402753
4,P60422,RPLB,P02918,MRCA,0.236,19402753


In [8]:
selected_nodes['sseqid'] = selected_nodes['sseqid'].str.split('|', expand = True)[1]
selected_nodes.set_index('sseqid', inplace = True)


In [9]:
edge['gene_one'] = edge['Protein A'].map(selected_nodes['qseqid'])
edge['gene_two'] = edge['Protein B'].map(selected_nodes['qseqid'])

edge.dropna(how = 'any', inplace = True) # we do not need interaction between genes that does not exist in our net

# drop useless columns
edge.drop(labels = ['Protein A', 'Protein B'], axis = 1, inplace = True)


In [10]:
edge.shape

(2059, 6)

In [11]:
edge.head()

,Gene A,Gene B,Score,PMID,gene_one,gene_two
85,Q46731,Q46731,0.554,10207011 10884228,562.57367.con.0062_15|562.57367,562.57367.con.0062_15|562.57367
99,UACT,RPSL,0.236,19402753,AYIO01000011_261|1328432.3,AYIO01000014_51|1328432.3
134,EMRA,EMRA,0.623,19805313 24561554,562.65030.con.0015_51|562.65030,562.65030.con.0015_51|562.65030
169,MUKB,RPSG,0.261,15690043,AYIO01000007_657|1328432.3,AYIO01000014_50|1328432.3
173,MURI,MURI,0.332,15690043 24627523 24561554,562.22860.con.0039_1|562.22860,562.22860.con.0039_1|562.22860


In [12]:
edge.to_csv('/home/hermuba/nas2/resist_subnet/mentha/mentha.mapped.csv')

In [13]:
import os
base_path = '/home/hermuba/nas2/resist_subnet/'
annotation_df = os.path.join(base_path, 'gold_anno.csv')
gold_anno = pd.read_csv(annotation_df, index_col = 0)
def PPV(computable, subset = None):
    if subset:
        subcomputable = computable.loc[(computable['target'].isin(subset))|(computable['source'].isin(subset))]
    else:
        subcomputable = computable.copy()
    grouped = subcomputable.groupby(by = ['goldstandard', 'prediction']).count()['target']
    
    try:
        tp = grouped[True, True]
        fp = grouped[False, True]
        PPV = tp/(tp+fp)
        
       
    except KeyError:
        PPV = 0
        
    return PPV

def coverage(computable, all_nodes):
    net = computable.loc[computable['prediction']]
    covered_nodes = set(net['source']).union(net['target']).intersection(all_nodes)
    
    return len(covered_nodes)/len(all_nodes)
    
def PPV_coverage(df, score_col, score_thres, gold_col = 'gosim', gold_thres = 0.6):
    ''' define goldstandard_score > gold_thres as Positive. d'''
    
    # write prediction
    computable = df.loc[(df[score_col].notnull())&(df[gold_col].notnull())]
    computable['prediction'] = df[score_col]>score_thres
    computable['goldstandard'] = df[gold_col]>gold_thres

    
    # total
    ppv = PPV(computable)
    cov = coverage(computable, set(gold_anno['gene_id']))
    
    # core
    core_genes = set(gold_anno.loc[gold_anno['core'], 'gene_id'])
    core_ppv = PPV(computable, subset = core_genes)
    core_cov = coverage(computable, core_genes)
    
    # acc
    acc_genes = set(gold_anno.loc[~gold_anno['core'], 'gene_id'])
    acc_ppv = PPV(computable, subset = acc_genes)
    acc_cov = coverage(computable, acc_genes)
    
    return ppv, core_ppv, acc_ppv, cov, core_cov, acc_cov

def try_thres(df, col, q = 300, interval = 10):
    tradeoff = []

    _, bins = pd.qcut(df[col], q = q, retbins = True, duplicates = 'drop')
    
    for bi in range(0, len(bins), interval):
        
        b = bins[bi]
        scores = PPV_coverage(df, col, b)
        tradeoff.append([b]+list(scores))
    return pd.DataFrame(tradeoff, columns = ['{}_score'.format(col), 'ppv', 'core_ppv', 'acc_ppv', 'cov', 'core_cov', 'acc_cov'])
    

In [14]:
# map edge to gene_id
edge['source'] = edge['gene_one'].map(gold_anno['gene_id'])
edge['target'] = edge['gene_two'].map(gold_anno['gene_id'])
edge['mentha'] = edge['Score']

In [15]:
# join gosim
gosim_df = pd.read_csv(os.path.join(base_path, 'gosim.csv'), sep = ' ', names = ['gene_id1', 'gene_id2', 'gosim'], skiprows = 1 )

In [ ]:
import networkx as nx
gosim = nx.from_pandas_edgelist(gosim_df, source = 'gene_id1', target = 'gene_id2',  edge_attr='gosim')
mentha = nx.from_pandas_edgelist(edge, edge_attr='mentha')
combined = nx.compose(mentha, gosim)
edges_prediction = nx.to_pandas_edgelist(combined)

In [ ]:
edges_prediction.head()

In [ ]:
net='mentha'
d = try_thres(edges_prediction, net)
d.to_csv(os.path.join(base_path, 'preformance', '{}.ppv_cov.csv'.format(net)))

In [ ]:
core_genes = gold_anno.loc[gold_anno['core'] == True]
def net_stat(G):
    # core
    number_of_core = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['core'] == True].index))
    # accessory
    number_of_acc = G.number_of_nodes() - number_of_core
    # hypothetical
    number_of_hypo = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['hypo_nr'] == True].index))
    # mobile
    number_of_mobile = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['is_aclame'] == True].index))
    # CARD genes
    number_of_card = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['is_card'] == True].index))
    # COG annotation
    number_of_cog = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['cog_ID'].notna()].index))
    return([G.number_of_nodes(), 
            G.number_of_edges(), 
            nx.algorithms.components.number_connected_components(G),
           number_of_core,
           number_of_acc,
           number_of_hypo,
           number_of_mobile,
           number_of_card,
           number_of_cog])



In [ ]:
net_stat(mentha)